In [2]:
import pandas as pd
dtest_dar = pd.read_excel('test_set_darija_8fev24.xlsx')
dtest_dar

,text,label,language
0,هادشي واعر,general_positive_feedback,ar
1,كيفاش نصلح طابعة عالقة؟,oodoos,ar
2,واش نقدر نلغي طلبية لشهادة ايداع جديدة؟,cancel_order,ar
3,عمليات مشبوهة فحسابي.,compromised_card,ar
4,تأجل الماتش، شنو لازم ندير؟,get_refund,ar
...,...,...,...
357,تقدر تسول الوكيل على ساعات دعم الزبائن؟,contact_customer_service,ar
358,كيفاش نعرف بلي تأميني مازال ساري؟,insurance,ar
359,بغيت نعدل المعلومات فبروفيل ديالي، كيفاش نديرها؟,edit_account,ar
360,تقدر تحاول تواجه مشاكلي بلا ما تعطيني جوابات ج...,general_negative_feedback,ar


In [3]:
intents = set(dtest_dar['label'])
intents

{'activate_my_card',
 'age_limit',
 'cancel_order',
 'cancel_transfer',
 'card_swallowed',
 'change_order',
 'compromised_card',
 'contact_customer_service',
 'contact_human_agent',
 'create_account',
 'deactivate_my_card',
 'delete_account',
 'edit_account',
 'exchange',
 'fee',
 'general_negative_feedback',
 'general_positive_feedback',
 'get_invoice',
 'get_refund',
 'idoos',
 'insurance',
 'loan',
 'lost_or_stolen_phone',
 'oodoos'}

In [4]:
len(intents)

24

In [1]:
from collections import Counter

def compute_metrics(ground_truth, predictions):
    unique_labels = set(ground_truth + predictions)
    true_positives = Counter()
    false_positives = Counter()
    false_negatives = Counter()
    
    for true_label, predicted_label in zip(ground_truth, predictions):
        if true_label == predicted_label:
            true_positives[true_label] += 1
        else:
            false_positives[predicted_label] += 1
            false_negatives[true_label] += 1
            
    precision = {}
    recall = {}
    f1_score = {}
    
    for label in unique_labels:
        precision[label] = true_positives[label] / (true_positives[label] + false_positives[label]) if (true_positives[label] + false_positives[label]) else 0
        recall[label] = true_positives[label] / (true_positives[label] + false_negatives[label]) if (true_positives[label] + false_negatives[label]) else 0
        f1_score[label] = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label]) if (precision[label] + recall[label]) else 0
    
    macro_precision = sum(precision.values()) / len(unique_labels)
    macro_recall = sum(recall.values()) / len(unique_labels)
    macro_f1_score = sum(f1_score.values()) / len(unique_labels)
    accuracy = sum(true_positives.values()) / len(ground_truth)
    
    return accuracy, macro_precision, macro_recall, macro_f1_score

# Example usage:
ground_truth = ['a', 'b', 'c', 'a', 'b', 'c']
predictions = ['a', 'b', 'c', 'c', 'b', 'a']
accuracy, precision, recall, f1_score = compute_metrics(ground_truth, predictions)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}")

Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666


In [5]:
meaning_of_intents={
    'activate_my_card': 'Initiate the use of a new banking card.',
    'age_limit': 'Inquire about the minimum age requirement for a service.',
    'cancel_order': 'Request to cancel a previously placed order.',
    'cancel_transfer': 'Request to cancel a previously initiated money transfer.',
    'card_swallowed': 'Report an ATM machine retaining a banking card.',
    'change_order': 'Modify details of a previously placed order.',
    'compromised_card': 'Report a banking card suspected of being at risk of fraud.',
    'contact_customer_service': 'Request for assistance from the bank’s customer service.',
    'contact_human_agent': 'Seek to speak with a live customer support agent.',
    'create_account': 'Initiate the process of opening a new bank account.',
    'deactivate_my_card': 'Disable a currently active banking card.',
    'delete_account': 'Request the closure of a bank account.',
    'edit_account': 'Make changes to the account information.',
    'exchange': 'Inquire about currency exchange services.',
    'fee': 'Question about the charges associated with a service.',
    'general_negative_feedback': 'Provide negative feedback on the overall service.',
    'general_positive_feedback': 'Provide positive feedback on the overall service.',
    'get_invoice': 'Request a bill or invoice for a transaction.',
    'get_refund': 'Request a return of funds for a transaction.',
    'insurance': 'Inquire about insurance products offered by the bank.',
    'loan': 'Request information on loan products.',
    'lost_or_stolen_phone': 'Report a lost or stolen phone linked to mobile banking.',
    'idoos': 'An intent not in the list of intents but within the banking domain, like asking for Western Union facilities.',
    'oodoos': 'An intent not in the list of intents and not related to banking, like asking the distance between the Earth and the Moon.'
}


In [30]:
prompt = """Context: You are an advanced banking chatbot designed for a Moroccan bank, equipped to assist customers with a range of inquiries and services related to banking. Your capabilities extend from handling basic account management to addressing complex service requests. Your primary objective is to accurately discern the customer's intent from their utterances, using the list of predefined intents to provide relevant assistance or guide them to the appropriate service channel.

Here is the list of all intents and their meanings:
- activate_my_card: Initiate the use of a new banking card.
- age_limit: Inquire about the minimum age requirement for a service.
- cancel_order: Request to cancel a previously placed order.
- cancel_transfer: Request to cancel a previously initiated money transfer.
- card_swallowed: Report an ATM machine retaining a banking card.
- change_order: Modify details of a previously placed order.
- compromised_card: Report a banking card suspected of being at risk of fraud.
- contact_customer_service: Request for assistance from the bank’s customer service.
- contact_human_agent: Seek to speak with a live customer support agent.
- create_account: Initiate the process of opening a new bank account.
- deactivate_my_card: Disable a currently active banking card.
- delete_account: Request the closure of a bank account.
- edit_account: Make changes to the account information.
- exchange: Inquire about currency exchange services.
- fee: Question about the charges associated with a service.
- general_negative_feedback: Provide negative feedback on the overall service.
- general_positive_feedback: Provide positive feedback on the overall service.
- get_invoice: Request a bill or invoice for a transaction.
- get_refund: Request a return of funds for a transaction.
- insurance: Inquire about insurance products offered by the bank.
- loan: Request information on loan products.
- lost_or_stolen_phone: Report a lost or stolen phone linked to mobile banking.
- idoos: An intent not in the list of intents but within the banking domain, like asking for Western Union facilities.
- oodoos: An intent not in the list of intents and not related to banking, like asking the distance between the Earth and the Moon.

When you receive the 17 utterances from a customer, analyze the content to determine the most applicable intents. Consider the context of banking practices in Morocco, including services and customer expectations.

Instructions:
1. Read the customer's utterances carefully.
2. Identify the most relevant intent for each utterance from the predefined list.
3. Return the detected intents in JSON format for easy parsing:

```
{"intents": ["intent1", "intent2", "intent3", "intent4", "intent5", ..., "intent17"]}
```

Make sure to return only one intent for each utterance. Select the intent that best matches the customer's query or service need for each of the five utterances. If an utterance does not fit any predefined intents or falls outside the banking domain, use "oodoos" for unrelated queries and "idoos" for banking-related queries not listed among the predefined intents.
Here are the five utterances:
"""


In [31]:
darija_texts = dtest_dar['text'].tolist()
batched_darija_texts = [darija_texts[k:k+17] for k in range(0,len(darija_texts),17)]

In [44]:
batched_darija_texts[:3]

[['هادشي واعر',
  'كيفاش نصلح طابعة عالقة؟',
  'واش نقدر نلغي طلبية لشهادة ايداع جديدة؟ ',
  'عمليات مشبوهة فحسابي. ',
  'تأجل الماتش، شنو لازم ندير؟',
  'شنو هي أحسن طريقة باش نتدربوا باش نعديوا ماراطون؟',
  'يمكن نلغي طلبية على رسوم رفض بطاقة الديون عند البيع؟ ',
  'عندي سؤال، أشنو  ندير باش نهدر مع خدمة الزبناء؟ ',
  'سول عند وكيل فين نستلم استرجاع',
  'بغيت معلومات، أشنو لازم ندير باش نتصل بخدمة الزبون؟ ',
  'شنو ندير إذا كارط ديالي تقلعها شباك الالي برا وقت العمل؟',
  'عاوني باش نلقى رسوم ديال البطاقة ',
  'علق الصراف الآلي معايا وما عرفتش شنو ندير.',
  'تقدر تقولي أشنو هو البريد الإلكتروني ديال خدمة الزبون؟ ',
  'شريت شي حاجة وبغيت نغير شراء ديالي ',
  'ساعدني نغلق حسابي',
  'شي واحد عنو نسخة من كرطتي ! أنا مازال عندي ديالي ولكن كاين لي سحب 500 جنيه نقدية، عاونوني! '],
 ['شنو أنواع الإنفاق الطبي المؤهل للاسترداد من حساب التوفير الصحي؟',
  'بغيت معلومات باش نهدر مع خدمة الزبون؟ ',
  'عافاك، تقدر تعيط لواحد الموظفين باش نعرف فين نحصَّل تعويض؟',
  'طلبت حاجة، شنو نقدر ندير باش نعدل 

In [9]:
len(darija_texts), len(batched_darija_texts)

(362, 73)

In [32]:
prompted_batched_dar = [prompt + "\n".join(x) for x in batched_darija_texts]

In [33]:
len(prompted_batched_dar)

22

In [15]:
from random import choice
print(choice(prompted_batched_dar))

Context: You are an advanced banking chatbot designed for a Moroccan bank, equipped to assist customers with a range of inquiries and services related to banking. Your capabilities extend from handling basic account management to addressing complex service requests. Your primary objective is to accurately discern the customer's intent from their utterances, using the list of predefined intents to provide relevant assistance or guide them to the appropriate service channel.

Here is the list of all intents and their meanings:
- activate_my_card: Initiate the use of a new banking card.
- age_limit: Inquire about the minimum age requirement for a service.
- cancel_order: Request to cancel a previously placed order.
- cancel_transfer: Request to cancel a previously initiated money transfer.
- card_swallowed: Report an ATM machine retaining a banking card.
- change_order: Modify details of a previously placed order.
- compromised_card: Report a banking card suspected of being at risk of fra

In [43]:
from openai import OpenAI
client = OpenAI(api_key="your_api_key_here")

def use_chatgpt(prompt):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    return response.choices[0].message.content

In [44]:
resp0 = use_chatgpt(prompted_batched_dar[0])
resp0

'{"intents": ["general_negative_feedback", "idoos", "cancel_order", "compromised_card", "idoos"]}'

In [46]:
from tqdm import tqdm
answers = [resp0]
for x in tqdm(prompted_batched_dar[1:]):
    answers.append(use_chatgpt(x))

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [02:33<00:00,  2.13s/it]


In [47]:
import pickle

with open('chatgpt_darija_intent_detection_on_test.pickle', 'wb') as handle:
    pickle.dump(answers, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [48]:
answers

['{"intents": ["general_negative_feedback", "idoos", "cancel_order", "compromised_card", "idoos"]}',
 '{"intents": ["oodoos", "cancel_order", "contact_customer_service", "get_refund", "contact_customer_service"]}',
 '{"intents": ["card_swallowed", "fee", "cancel_order", "contact_customer_service", "change_order"]}',
 '{"intents": ["delete_account", "activate_my_card", "get_refund", "contact_customer_service", "get_invoice"]}',
 '{"intents": ["change_order", "contact_human_agent", "loan", "compromised_card", "general_negative_feedback"]}',
 '{"intents": ["compromised_card", "get_refund", "card_swallowed", "get_refund", "deactivate_my_card"]}',
 '{"intents": ["lost_or_stolen_phone", "insurance", "activate_my_card", "get_invoice", "delete_account"]}',
 '{"intents": ["change_order", "idoos", "change_order", "insurance", "create_account"]}',
 '{"intents": ["cancel_order", "loan", "compromised_card", "delete_account", "loan"]}',
 '{"intents": ["get_refund", "cancel_order", "lost_or_stolen_ph

In [52]:
predicted_intents_dar = []
for x in answers:
    predicted_intents_dar.extend(eval(x)['intents'])


In [53]:
predicted_intents_dar

['general_negative_feedback',
 'idoos',
 'cancel_order',
 'compromised_card',
 'idoos',
 'oodoos',
 'cancel_order',
 'contact_customer_service',
 'get_refund',
 'contact_customer_service',
 'card_swallowed',
 'fee',
 'cancel_order',
 'contact_customer_service',
 'change_order',
 'delete_account',
 'activate_my_card',
 'get_refund',
 'contact_customer_service',
 'get_invoice',
 'change_order',
 'contact_human_agent',
 'loan',
 'compromised_card',
 'general_negative_feedback',
 'compromised_card',
 'get_refund',
 'card_swallowed',
 'get_refund',
 'deactivate_my_card',
 'lost_or_stolen_phone',
 'insurance',
 'activate_my_card',
 'get_invoice',
 'delete_account',
 'change_order',
 'idoos',
 'change_order',
 'insurance',
 'create_account',
 'cancel_order',
 'loan',
 'compromised_card',
 'delete_account',
 'loan',
 'get_refund',
 'cancel_order',
 'lost_or_stolen_phone',
 'contact_customer_service',
 'deactivate_my_card',
 'insurance',
 'general_positive_feedback',
 'general_negative_feedback

In [56]:
accuracy, precision, recall, f1_score = compute_metrics(ground_truth, predicted_intents_dar)
accuracy, precision, recall, f1_score

(0.5248618784530387, 0.5337524070132766, 0.5005717060717061, 0.505209988836986)